In [1]:
from math import inf
from copy import deepcopy

In [2]:
class Logger:
    def __init__(self):
        self.on = True
    def log(*args, **kwargs):
        self = args[0]
        if self.on:
            print(*args[1:], **kwargs)
logger = Logger()
logger.log('Hello, World!')
logger.on = False
logger.log('Hallo, wereld!')
logger.on = True
logger.log('Hola, ¡mundo!')


Hello, World!
Hola, ¡mundo!


In [3]:
class Node:
    def __init__(self, name : str, graph, distance=inf):
        self.name = str(name).upper()
        self.distance = distance
        self.visited = False
        self.graph = graph
        self.shortest_path_prev_node = None
    def __str__(self):
        if self.distance == inf:
            name =  f'Node({self.name})'
        else:
            name = f'Node({self.name}, d={self.distance})'
        if self.visited:
            name = f'\033[92m{name}\033[0m'
        elif self.distance != inf:
            name = f'\033[96m{name}\033[0m'
        return name
    def __repr__(self):
        return self.__str__()
    def edges(self, no_visited_nodes=True):
        connecting_edges =  [edge for edge in self.graph.edges if edge.node_start == self]
        if not no_visited_nodes:
            return connecting_edges
        return [edge for edge in connecting_edges if not edge.node_end.visited]

In [4]:
class Edge:
    def __init__(self, node_start : Node, node_end : Node, weight=inf):
        self.node_start = node_start
        self.node_end = node_end
        self.weight = weight
    
    def __str__(self):
        return f'Edge({self.node_start.name} → {self.node_end.name})'
    def __repr__(self):
        return self.__str__()

In [5]:
class Graph:
    def __init__(self):
        self.edges = []
        self.nodes = []

    
    def get_smallest_distance_unvisited_node(self):
        unvisited_nodes =  [node for node in self.nodes if not node.visited]
        return min(unvisited_nodes, key=lambda node: node.distance)

    def get_node(self, name):
        index = self.get_node_names().index(name)
        return self.nodes[index]
    
    def get_node_names(self):
        return [node.name for node in self.nodes]
    
    def add_node(self, name):
        node = Node(name, graph=self)
        self.nodes.append(node)
        return node
    
    def get_or_add_node(self, name):
        try:
            node_index = self.get_node_names().index(name)
        except ValueError:
            node_index = None
        
        if node_index is None:
            node = self.add_node(name)
        else:
            node = self.nodes[node_index]
        
        return node


    def add_edge(self, node_start_name, node_end_name, weight=inf, directed=True):
        node_start = self.get_or_add_node(node_start_name)
        node_end = self.get_or_add_node(node_end_name)

        self.edges.append(Edge(node_start, node_end, weight=weight))
        
        if not directed:
            self.edges.append(Edge(node_end, node_start, weight=weight))

In [10]:
graph = Graph()
graph.add_edge('A', 'C', weight=3, directed=False)
graph.add_edge('A', 'F', weight=2, directed=False)
graph.add_edge('C', 'F', weight=2, directed=False)
graph.add_edge('C', 'D', weight=4, directed=False)
graph.add_edge('C', 'E', weight=1, directed=False)
graph.add_edge('F', 'E', weight=3, directed=False)
graph.add_edge('D', 'B', weight=1, directed=False)
graph.add_edge('E', 'B', weight=2, directed=False)
graph.add_edge('F', 'B', weight=6, directed=False)
graph.add_edge('G', 'B', weight=2, directed=False)
graph.add_edge('F', 'G', weight=5, directed=False)

starting_node = graph.get_node('A')
finish_node = graph.get_node('B')

print(starting_node)
print(starting_node.edges())
print(graph.nodes)
print(graph.edges)

Node(A)
[Edge(A → C), Edge(A → F)]
[Node(A), Node(C), Node(F), Node(D), Node(E), Node(B), Node(G)]
[Edge(A → C), Edge(C → A), Edge(A → F), Edge(F → A), Edge(C → F), Edge(F → C), Edge(C → D), Edge(D → C), Edge(C → E), Edge(E → C), Edge(F → E), Edge(E → F), Edge(D → B), Edge(B → D), Edge(E → B), Edge(B → E), Edge(F → B), Edge(B → F), Edge(G → B), Edge(B → G), Edge(F → G), Edge(G → F)]


In [11]:
def get_shortest_path(graph, starting_node, finish_node):
    starting_node.distance = 0
    starting_node.visited = True
    current_node = starting_node
    logger.log(f'Starting with {starting_node}')
    while current_node != finish_node:
        
        for edge in current_node.edges(no_visited_nodes=False):
            distance_via_current_node = current_node.distance + edge.weight
            if distance_via_current_node < edge.node_end.distance:
                edge.node_end.distance = distance_via_current_node
                edge.node_end.shortest_path_prev_node = current_node

            # edge.node_end.distance = min(edge.node_end.distance, current_node.distance + edge.weight)
        
        current_node.visited = True
        # next_node = min(current_node.edges(), key=lambda edge: edge.node_end.distance).node_end
        next_node = graph.get_smallest_distance_unvisited_node()
        logger.log(f'State: {graph.nodes}')
        logger.log(f'Smallest unvisited {graph.get_smallest_distance_unvisited_node()}')
        logger.log(f'Going to {next_node}')
        logger.log()
        current_node = next_node

    shortest_path_reversed = list()
    node = finish_node
    while node != starting_node:
        shortest_path_reversed.append(node)
        node = node.shortest_path_prev_node
    shortest_path_reversed.append(starting_node)
    shortest_path = shortest_path_reversed[::-1]
    return shortest_path
        

shortest_path = get_shortest_path(graph, starting_node, finish_node)
print(' → '.join([node.name for node in shortest_path]))


Starting with Node(A, d=0)
State: [Node(A, d=0), Node(C, d=3), Node(F, d=2), Node(D), Node(E), Node(B), Node(G)]
Smallest unvisited Node(F, d=2)
Going to Node(F, d=2)

State: [Node(A, d=0), Node(C, d=3), Node(F, d=2), Node(D), Node(E, d=5), Node(B, d=8), Node(G, d=7)]
Smallest unvisited Node(C, d=3)
Going to Node(C, d=3)

State: [Node(A, d=0), Node(C, d=3), Node(F, d=2), Node(D, d=7), Node(E, d=4), Node(B, d=8), Node(G, d=7)]
Smallest unvisited Node(E, d=4)
Going to Node(E, d=4)

State: [Node(A, d=0), Node(C, d=3), Node(F, d=2), Node(D, d=7), Node(E, d=4), Node(B, d=6), Node(G, d=7)]
Smallest unvisited Node(B, d=6)
Going to Node(B, d=6)

Trafersing: Node(B, d=6)
Node(E, d=4)
Trafersing: Node(E, d=4)
Node(C, d=3)
Trafersing: Node(C, d=3)
Node(A, d=0)
A → C → E → B
